In [1]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('./vseros-rank-otb/train_df.tsv',sep='\t')
df.head(5)

,Unnamed: 0,community_id,description,customers_count,messages_count,type,region_id,themeid,business_category,business_parent,customer_id,status,join_request_date
0,0,00dbc35ec26c5b6452a3259194b2d2f74eae7141d2bc75...,"Ванга сказала: ""Выживет пчела - выживет челове...",2966,1,7,NaN,NaN,NaN,NaN,947224211267aefcc2e3e9c524fdf46ce329bc638e8bf1...,A,NaN
1,1,06107f1eae304c45d3e6324cc86f7d39662793a53b159b...,NaN,8982,2,7,1.042403e+10,246.0,NaN,NaN,b935c3390d82df612de19591d2dd16e1eff8e660746bb3...,A,NaN
2,2,06107f1eae304c45d3e6324cc86f7d39662793a53b159b...,NaN,8982,2,7,1.042403e+10,246.0,NaN,NaN,ff873dee9234ee290d92eb6c15d7179aabf0653dbed3b3...,A,NaN
3,3,06107f1eae304c45d3e6324cc86f7d39662793a53b159b...,NaN,8982,2,7,1.042403e+10,246.0,NaN,NaN,737ef0e849feda6cdfd350d19fcedaf17aebb757c0a1fc...,A,NaN
4,4,097e4dae68029ecc3c6183ed4386aff71dcac227cbaff7...,Читать внимательно!!! Причины попадания в чёрн...,1035,1,7,1.041779e+10,NaN,FAN_CLUB,BLOG,780fb8c178af972826ac0f84e42ac4c6c1dcdeeec0c4dd...,A,NaN


In [3]:
df = df.sample(frac=1,random_state=56)

In [4]:
df_rec_pair = df[['customer_id','community_id']].copy()

In [5]:
df_rec_pair['pair'] = df_rec_pair['customer_id'] + '[SEP]' + df_rec_pair['community_id']
counts = df_rec_pair.groupby('pair')['customer_id'].transform('count')
df_rec_pair['count_db'] = counts
df_rec_pair = df_rec_pair.drop_duplicates()

In [7]:
df_rec_pair.isnull().sum()

customer_id     0
community_id    0
pair            0
count_db        0
dtype: int64

In [9]:
df = df.loc[df_rec_pair.index]

In [10]:
df['count_db'] = df_rec_pair['count_db']

In [11]:
df['description'] = df['description'].isnull()

In [13]:
df['pair'] = df['customer_id'] + '[SEP]' + df['community_id']

In [14]:
df.to_parquet('all_train_df.parquet')

In [15]:
cust_group = df[['Unnamed: 0','customer_id']].groupby('customer_id').agg(['count',lambda x:[x]])

In [16]:
cust_group.head(5)

Unnamed: 0  \
                                                        count   
customer_id                                                     
000004017025a08c984916e57658b1ab64deb4172db1634...         11   
0000079a765e2fd7fbb3d8c6968474d7a40cc9ada4c9ec4...         13   
000015dccbb94f6926bfedc1578d6207ab10c6580fb121c...          1   
00001a206a3508a172a7be2b19c450b72adfc46239e1ed6...          1   
0000208fc1af439a84a92acd34352268ab169b1a6536e6b...          1   

                                                                                                       
                                                                                           <lambda_0>  
customer_id                                                                                            
000004017025a08c984916e57658b1ab64deb4172db1634...  [[8154644, 9374214, 9756328, 10252035, 8512732...  
0000079a765e2fd7fbb3d8c6968474d7a40cc9ada4c9ec4...  [[7741667, 7978340, 8177058, 8582211, 9201207,...  
000015dccbb94f6926bfedc1578d6207ab10c6580fb121c...                                       [[14976946]]  
00001a206a3508a172a7be2b19c450b72adfc46239e1ed6...                                        [[1802565]]  
0000208fc1af439a84a92acd34352268ab169b1a6536e6b...                                         [[488198]]

In [17]:
test_customers = cust_group[cust_group['Unnamed: 0']['count'] >= 8].sample(100_000,random_state=56)

In [18]:
test_idxes = test_customers['Unnamed: 0']['<lambda_0>']

In [19]:
train_gbm,all_test = train_test_split(test_idxes,test_size=0.3,random_state=56)

In [20]:
real_test_index = []
for i in tqdm(all_test.values):
  real_test_index += i[0][:5].tolist()

  0%|          | 0/30000 [00:00<?, ?it/s]

/tmp/ipykernel_63/1962256308.py:3: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  real_test_index += i[0][:5].tolist()


In [21]:
np.save('test_index.npy',real_test_index)

In [23]:
train_gbm_index = []
for i in tqdm(train_gbm.values):
  train_gbm_index += i[0][:5].tolist()

  0%|          | 0/70000 [00:00<?, ?it/s]

/tmp/ipykernel_63/3320461818.py:3: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  train_gbm_index += i[0][:5].tolist()


In [24]:
np.save('train_gbm_index.npy',train_gbm_index)

In [25]:
train_gbm_index[0]

12390705